In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
test_df=pd.read_csv('./output/train_hypothesis_evidences.csv')
test_df

,hypothesis,evidences,label
0,All the primary trial participants do not rece...,"[""INTERVENTION 1: "", "" Diagnostic (FLT PET)"",...",Contradiction
1,"Patients with Platelet count over 100,000/mm¬¨...","["" PATIENT CHARACTERISTICS:"", "" ANC 1,500/m...",Contradiction
2,Heart-related adverse events were recorded in ...,"[""Adverse Events 1:"", "" Supraventricular tach...",Entailment
3,Adult Patients with histologic confirmation of...,"[""Inclusion Criteria:"", "" Patients with histo...",Contradiction
4,Laser Therapy is in each cohort of the primary...,"[""INTERVENTION 1: "", "" Laser Therapy Alone"", ...",Contradiction
...,...,...,...
1695,"Adequate blood, kidney, and hepatic function a...","[""Inclusion Criteria:"", "" Postmenopausal wome...",Entailment
1696,The Ridaforolimus + Dalotuzumab + Exemestane g...,"[""Outcome Measurement: "", "" 1. Progression-fr...",Contradiction
1697,The only difference between the interventions ...,"[""INTERVENTION 1: "", "" Prone"", ""Prone positio...",Entailment
1698,Patients must have a white blood cell count ab...,"["" WBC > 1,500/mm\u00b3""]",Entailment


In [3]:
hypothesis_lst=test_df['hypothesis'].values.tolist()
len(hypothesis_lst)

1700

In [4]:
evidence_lst=test_df['evidences'].apply(lambda l:' '.join(json.loads(l))).values.tolist()
len(evidence_lst)

1700

In [5]:
label2id={"Contradiction":0,"Entailment":1}
label_lst=test_df['label'].apply(lambda x:label2id[x]).values.tolist()
len(label_lst)

1700

In [6]:
import json
with open('output/all_evidences.json','r') as f:
    all_evidences = json.load(f)
evidence2id=dict([(all_evidences[i],i) for i in range(len(all_evidences))])

from nltk.tokenize import word_tokenize
tokenized_corpus = [word_tokenize(doc) for doc in all_evidences]

from rank_bm25 import BM25Okapi
bm25 = BM25Okapi(tokenized_corpus)

In [7]:
import numpy as np
scores=[]
for hypothesis,evidences in test_df[['hypothesis','evidences']].values:
    doc_scores = bm25.get_scores(word_tokenize(hypothesis))
    doc_scores = (doc_scores-np.min(doc_scores))/(np.max(doc_scores)-np.min(doc_scores))
    scores.append(np.mean([doc_scores[evidence2id[evi]] for evi in json.loads(evidences)]))
# scores

In [8]:
from sklearn.metrics import average_precision_score,f1_score,precision_score,recall_score,accuracy_score

y_true=label_lst
results=[]
for thre in np.arange(0.0, 1.0, 0.1):
    y_prob=scores
    y_pred=[1 if a>thre else 0 for a in y_prob]
    results.append([
        thre,
        average_precision_score(y_true,y_prob),
        f1_score(y_true,y_pred),
        precision_score(y_true,y_pred),
        recall_score(y_true,y_pred),
        accuracy_score(y_true,y_pred)
    ])

import pandas as pd

pd.DataFrame(results,columns=['threshold','AVG_PREC','F1','PREC','REC','ACC'])

,threshold,AVG_PREC,F1,PREC,REC,ACC
0,0.0,0.500468,0.631757,0.492754,0.880000,0.487059
1,0.1,0.500468,0.512115,0.481366,0.547059,0.478824
2,0.2,0.500468,0.378058,0.511022,0.300000,0.506471
3,0.3,0.500468,0.244080,0.540323,0.157647,0.511765
4,0.4,0.500468,0.150754,0.517241,0.088235,0.502941
5,0.5,0.500468,0.070252,0.524590,0.037647,0.501765
6,0.6,0.500468,0.040770,0.545455,0.021176,0.501765
7,0.7,0.500468,0.025316,0.578947,0.012941,0.501765
8,0.8,0.500468,0.016222,0.538462,0.008235,0.500588
9,0.9,0.500468,0.007001,0.428571,0.003529,0.499412
